<a href="https://colab.research.google.com/github/DinDin23/Spotify-Songs-Suggestion-Model/blob/main/Spotify_Songs_Suggestion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify Suggestion Model V1

## Infrastructure

In [ ]:
import sys
!{sys.executable} -m pip install spotipy

In [ ]:
username = "dineshsp"
cid = "b858cb986e874b8ca3e1a437a81b3dcd" 
secret = "8d7a73e652544e20b5b8ccebe4d8f03c"
good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
bad_playlist_id = "53FPslFSIAGvVyERcNG4J3"
source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from statistics import mean, stdev
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

## Data Collection

In [ ]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [ ]:
good_playlist = sp.user_playlist(username, good_playlist_id)
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"]

while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)

good_ids = []
for i in range(len(good_songs)):
    good_ids.append(good_songs[i]['track']['id'])
print(good_songs)

In [ ]:
bad_playlist = sp.user_playlist(username, bad_playlist_id)
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"]

while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)

bad_ids = []
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])
print(len(bad_ids))
print(bad_ids)

In [ ]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

In [ ]:
training_features = []

for i in range(0, len(good_ids), 50):
    audio_features = sp.audio_features(good_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 1

for i in range(0, len(bad_ids), 50):
    audio_features = sp.audio_features(bad_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

In [ ]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        source_features.append(track)
        source_features[-1]['target'] = 0  

print(len(source_features))
print(source_features)

## Data Preparation

In [ ]:
trainingData = pd.DataFrame(training_features)

In [ ]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.482,0.786,7,-5.735,1,0.0320,0.00321,0.000089,0.4460,0.7930,126.755,audio_features,4DMKwE2E2iYDKY01C335Uw,spotify:track:4DMKwE2E2iYDKY01C335Uw,https://api.spotify.com/v1/tracks/4DMKwE2E2iYD...,https://api.spotify.com/v1/audio-analysis/4DMK...,323000,4,1
1,0.377,0.681,7,-8.039,1,0.0298,0.00088,0.002300,0.0504,0.2850,108.789,audio_features,1QEEqeFIZktqIpPI4jSVSF,spotify:track:1QEEqeFIZktqIpPI4jSVSF,https://api.spotify.com/v1/tracks/1QEEqeFIZktq...,https://api.spotify.com/v1/audio-analysis/1QEE...,285133,4,1
2,0.333,0.927,9,-8.550,0,0.0733,0.00290,0.000208,0.2970,0.3850,141.466,audio_features,5QTxFnGygVM4jFQiBovmRo,spotify:track:5QTxFnGygVM4jFQiBovmRo,https://api.spotify.com/v1/tracks/5QTxFnGygVM4...,https://api.spotify.com/v1/audio-analysis/5QTx...,308120,4,1
3,0.316,0.320,9,-13.045,1,0.0387,0.40800,0.000067,0.0925,0.2940,74.163,audio_features,05GhiSRMv34B1N6BsixbmL,spotify:track:05GhiSRMv34B1N6BsixbmL,https://api.spotify.com/v1/tracks/05GhiSRMv34B...,https://api.spotify.com/v1/audio-analysis/05Gh...,250933,3,1
4,0.341,0.293,9,-12.408,1,0.0356,0.75600,0.000023,0.0833,0.1780,93.294,audio_features,2V0yBHMWB8GoMm80AXH4ot,spotify:track:2V0yBHMWB8GoMm80AXH4ot,https://api.spotify.com/v1/tracks/2V0yBHMWB8Go...,https://api.spotify.com/v1/audio-analysis/2V0y...,249213,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.567,0.463,6,-11.669,0,0.5040,0.73200,0.000000,0.7090,0.3240,126.982,audio_features,40IQooNkPRVtn4zlE3ZFpA,spotify:track:40IQooNkPRVtn4zlE3ZFpA,https://api.spotify.com/v1/tracks/40IQooNkPRVt...,https://api.spotify.com/v1/audio-analysis/40IQ...,496173,4,0
345,0.554,0.498,9,-8.866,1,0.0685,0.23000,0.000006,0.7950,0.4130,79.635,audio_features,5hVghJ4KaYES3BFUATCYn0,spotify:track:5hVghJ4KaYES3BFUATCYn0,https://api.spotify.com/v1/tracks/5hVghJ4KaYES...,https://api.spotify.com/v1/audio-analysis/5hVg...,190067,4,0
346,0.598,0.252,2,-15.086,1,0.0644,0.19400,0.133000,0.0992,0.0352,120.064,audio_features,4S7YHmlWwfwArgd8LfSPud,spotify:track:4S7YHmlWwfwArgd8LfSPud,https://api.spotify.com/v1/tracks/4S7YHmlWwfwA...,https://api.spotify.com/v1/audio-analysis/4S7Y...,238307,4,0
347,0.621,0.601,6,-5.616,0,0.1480,0.05220,0.000000,0.4600,0.4570,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5,0


In [ ]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [ ]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [ ]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
247,0.735,-2.939,0.369,0.525,0.918000,0.65300,6,0.0441,157777
32,0.436,-3.375,0.459,0.792,0.000000,0.01790,9,0.0341,240533
301,0.692,-8.465,0.463,0.521,0.000000,0.30100,10,0.3300,292987
169,0.769,-5.591,0.612,0.569,0.000000,0.07370,1,0.0786,216946
99,0.313,-6.215,0.300,0.882,0.338000,0.03460,0,0.0355,431093
...,...,...,...,...,...,...,...,...,...
235,0.931,-9.127,0.376,0.387,0.000000,0.08800,1,0.4120,179405
56,0.329,-8.465,0.323,0.673,0.109000,0.02390,3,0.0296,419933
2,0.333,-8.550,0.385,0.927,0.000208,0.00290,9,0.0733,308120
8,0.392,-9.961,0.228,0.402,0.000000,0.28800,0,0.0536,354320


In [ ]:
y_train

In [ ]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
158,0.776,-9.143,0.294,0.388,0.000000,0.147000,0,0.2820,161000
298,0.639,-8.145,0.235,0.549,0.000000,0.035800,10,0.2670,193341
299,0.647,-6.023,0.577,0.706,0.000000,0.264000,6,0.1370,182338
195,0.846,-8.059,0.441,0.538,0.000000,0.027200,1,0.0576,208160
183,0.721,-6.847,0.614,0.766,0.000000,0.091300,7,0.3240,175294
259,0.624,-4.261,0.141,0.659,0.000000,0.246000,4,0.2550,176328
330,0.886,-8.054,0.408,0.479,0.000000,0.077000,4,0.3340,158692
196,0.473,-6.474,0.615,0.461,0.000000,0.149000,6,0.3330,234213
69,0.456,-3.551,0.532,0.863,0.053800,0.133000,3,0.0894,301227
121,0.463,-7.950,0.550,0.943,0.213000,0.030300,10,0.0419,375307


In [ ]:
y_test

## Model Evaluation

In [ ]:
linreg = LinearRegression()
linreg.fit(x_train, y_train)
linreg_pred = linreg.predict(x_test)

print('Mean squared error: %.2f' % mean_squared_error(y_test, linreg_pred))
linreg_pred

In [ ]:
knn = KNeighborsClassifier(3)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)

score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, knn_pred))

In [ ]:
dtc = DecisionTreeClassifier(min_samples_split=100)
dtc.fit(x_train, y_train)
dtc_pred = dtc.predict(x_test)

score = accuracy_score(y_test, dtc_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, dtc_pred))

In [ ]:
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))

In [ ]:
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)

score = accuracy_score(y_test, ada_pred) * 100
print("Accuracy using Ada Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, ada_pred))

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=2)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

In [ ]:
model = ada

## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.511,-10.407,0.498,0.942,0.000019,0.1740,0,0.0429,393667
1,0.454,-4.126,0.488,0.948,0.000000,0.0142,5,0.0831,154133
2,0.535,-11.159,0.917,0.810,0.074700,0.0110,2,0.0319,158573
3,0.298,-9.905,0.655,0.704,0.000488,0.1880,7,0.0366,302120
4,0.660,-6.610,0.678,0.816,0.018100,0.1680,2,0.0299,263040
...,...,...,...,...,...,...,...,...,...
173,0.572,-8.190,0.662,0.759,0.000000,0.0303,10,0.0583,207849
174,0.683,-7.721,0.290,0.571,0.061600,0.0335,1,0.0286,261001
175,0.798,-4.534,0.782,0.662,0.000091,0.0868,5,0.0251,180000
176,0.838,-6.525,0.971,0.919,0.000125,0.0533,2,0.0445,264000


In [ ]:
pred = model.predict(sourceData)

In [ ]:
source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")

i = 0
for prediction in pred:
    if prediction == 1:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Shout, By: Tears For Fears
Song: Fully Alive, By: Flyleaf
Song: Broken, By: Tears For Fears
Song: Head Over Heels / Broken, By: Tears For Fears
Song: Don't You (Forget About Me), By: Simple Minds
Song: Animal, By: Def Leppard
Song: Mad World (Feat. Michael Andrews), By: Gary Jules
Song: Thing For You, By: Randall Bramblett
Song: More Than This, By: Roxy Music
Song: cherry blossom, By: slenderbodies
Song: out 4 a breather, By: dad sports
Song: Two Hearts, By: Zola Blood
Song: Nightfall, By: Mimi Page
Song: Secunda, By: Jeremy Soule
Song: The World Retreats, By: David O'Dowda
Song: Atlas - From “The Hunger Games: Catching Fire” Soundtrack, By: Coldplay
Song: Weird Fishes/ Arpeggi, By: Radiohead
Song: Tur, By: Marco Egeberg
Song: With you, By: Cécile Corbel
Song: Exploration, By: Bruno Coulais
Song: Broken Sleep, By: Agnes Obel
Song: Midnight, By: Coldplay
Song: Blood Pressurize, By: RemoteTreeChildren
Song: Under The Pressure, By: The War On Drugs
Song: Round and Round, By